In [ ]:
import json
import pandas as pd
from scipy.stats import ttest_ind, ttest_ind_from_stats

In [ ]:
#store transcript into a pandas dataframes, create a new dataframe that contains only report_id and features 
df = pd.read_csv('../DB_manual_v34.csv')

df_feat = df.loc[:, "ADJP_->_JJ":]
df_feat["report_id"] = df.loc[:, "report_id"]

In [ ]:
#store feature names into a list
feature_list = df.columns.tolist()[29:]

In [ ]:
#create dictionary with AD/HC diagnosis
diagnosis = {}
for index, row in df.iterrows():
    report_id = row['report_id']
    label = json.loads(row["diagnoses"].replace("[", "").replace("]", ""))
    diagnosis.update({report_id:label['name']})
    #diagnosis_list.append(diagnosis)
print(diagnosis)

In [ ]:
#loop through the dataframe
t_values = []
p_values = []
for column in df_feat.columns:
    if column is not "report_id":
        ad_feat_list = []
        hc_feat_list = []
        for index, row in df_feat.iterrows():
            label = diagnosis.get(row['report_id'])
            if label == 'hc':
                hc_feat = row[column]
                hc_feat_list.append(hc_feat)
            else:
                ad_feat = row[column]
                ad_feat_list.append(ad_feat)
        t, p = ttest_ind(ad_feat_list, hc_feat_list, equal_var = False, nan_policy='omit')
        t_values.append(t)
        p_values.append(p)

In [ ]:
#create a new dataframe to collect p values for all the features
df_stats = pd.DataFrame(feature_list, columns = ['features'])
df_stats['p value'] = p_values

In [ ]:
#identify features of high significance
significant_feat = []
for index, row in df_stats.iterrows():
    if row['p value'] < 0.05:
        significant_feat.append(row['features'])
        print(significant_feat)